# Import

In [1]:
import duckdb
import requests
import glob
import os
import gc
import shutil
import time
from pathlib import Path
import pandas as pd
import os
import dotenv
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime
from google import genai

In [2]:
dotenv.load_dotenv()

True

# Load Data

In [3]:
## Import Block 
## SYSTEM SETUP & HELPER FUNCTIONS (You are free to edit as needed)
# --- 🛠️ SYSTEM SETUP & HELPER FUNCTIONS (Do not edit unless necessary) ---

# 1. Imports
import duckdb
import requests
import glob
import os
import gc
import shutil
import time
from pathlib import Path

# Detect Environment
try:
    from IPython.display import display, HTML
    IN_NOTEBOOK = True
except ImportError:
    IN_NOTEBOOK = False

# 2. Database Wrapper Class
class DuckDBWrapper:
    def __init__(self, duckdb_path=None):
        self.db_path = Path(duckdb_path).resolve() if duckdb_path else None
        self.con = None
        self.registered_tables = [] 
    
    def connect(self):
        """Establishes connection and loads HTTPFS for remote files."""
        if self.con: return
        try:
            if self.db_path:
                self.con = duckdb.connect(str(self.db_path), read_only=False)
            else:
                self.con = duckdb.connect(database=':memory:', read_only=False)
            self.con.execute("INSTALL httpfs; LOAD httpfs;")
        except Exception as e:
            print(f"❌ Connection Failed: {e}")
            raise e

    def close(self):
        if self.con:
            try: self.con.close()
            except: pass
            self.con = None

    def register_data_view(self, paths, table_names):
        """Creates virtual views for Parquet/CSV/JSON files (Zero-Copy)."""
        if not self.con: self.connect()
        if len(paths) != len(table_names): raise ValueError("Length mismatch")
        
        for path, table_name in zip(paths, table_names):
            path_str = str(path)
            if not glob.glob(path_str) and not os.path.exists(path_str): continue
            try:
                # Logic: Detect filetype and use appropriate DuckDB reader
                if ".parquet" in path_str: query = f"CREATE OR REPLACE VIEW {table_name} AS SELECT * FROM read_parquet('{path_str}')"
                elif ".csv" in path_str: query = f"CREATE OR REPLACE VIEW {table_name} AS SELECT * FROM read_csv_auto('{path_str}')"
                elif ".json" in path_str: query = f"CREATE OR REPLACE VIEW {table_name} AS SELECT * FROM read_json_auto('{path_str}')"
                else: continue
                
                self.con.execute(query)
                if table_name not in self.registered_tables: self.registered_tables.append(table_name)
            except Exception as e: print(f"❌ Error registering {table_name}: {e}")

    def run_query(self, sql_query, show_results=False):
        """Executes SQL. Returns DataFrame. Displays scrollable HTML if show_results=True."""
        if not self.con: self.connect()
        import polars as pl 
        
        try:
            arrow_table = self.con.execute(sql_query).arrow()
            df = pl.DataFrame(arrow_table)
            
            if show_results:
                if IN_NOTEBOOK:
                    # 💡 UI FEATURE: Pandas for reliable HTML Table rendering
                    pdf = df.head(1000).to_pandas()
                    table_html = pdf.to_html(index=False, border=0, classes=["dataframe"])
                    scrollable_div = f"""
                    <div style="max-height: 400px; overflow-y: auto; overflow-x: auto; border: 1px solid #444;">
                        <style>.dataframe thead th {{ position: sticky; top: 0; background: #222; color: white; }}</style>
                        {table_html}
                    </div>
                    """
                    display(HTML(scrollable_div))
                else:
                    self._print_simple_table(df)
                return None
            return df
        except Exception as e:
            print(f"❌ Query Failed: {e}")
            return None

    def show_tables(self):
        """Show tables using the Brighter Rich style."""
        df = self.run_query("SELECT table_name, table_type FROM information_schema.tables WHERE table_schema='main'", show_results=False)
        if df is not None:
            self._print_fancy_table(df, title="📂 Database Assets")

    def show_schema(self, table_name):
        """Show schema using the Brighter Rich style."""
        query = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'"
        df = self.run_query(query, show_results=False)
        if df is not None:
            self._print_fancy_table(df, title=f"📋 Schema: {table_name}")

    def _print_fancy_table(self, df, title):
        """A High-Contrast, Bright table for metadata."""
        from rich.console import Console; from rich.table import Table; from rich import box
        console = Console()
        table = Table(title=title, title_style="bold bright_yellow", header_style="bold bright_white", box=box.ROUNDED, show_lines=True, border_style="bright_black")
        for col in df.columns: table.add_column(col, style="bright_cyan", justify="left")
        for row in df.iter_rows(named=True): table.add_row(*[str(v) for v in row.values()])
        console.print(table)

    def _print_simple_table(self, df):
        """Fallback table for terminal."""
        from rich.console import Console; from rich.table import Table; from rich import box
        console = Console()
        table = Table(title="Query Results", box=box.SIMPLE, show_lines=False)
        for col in df.columns: table.add_column(col, style="dim", no_wrap=True, overflow="ellipsis", max_width=30)
        for row in df.head(10).iter_rows(named=True): table.add_row(*[str(v) if v is not None else "" for v in row.values()])
        console.print(table)

    def export(self, data, file_name, file_type="csv", output_dir="../data/exports"):
        """Smart Export."""
        if isinstance(data, str):
            print(f"⏳ Running query for export: '{file_name}'...")
            df = self.run_query(data, show_results=False)
        else:
            df = data
        if df is None or df.height == 0: print("⚠️ Export skipped (Empty/None)"); return

        full_path = Path(output_dir) / f"{file_name}.{file_type}"
        full_path.parent.mkdir(parents=True, exist_ok=True)
        try:
            if file_type == "parquet": df.write_parquet(str(full_path))
            elif file_type == "csv": df.write_csv(str(full_path))
            elif file_type == "json": df.write_ndjson(str(full_path))
            else: print(f"❌ Unknown format: {file_type}"); return
            print(f"✅ Exported {df.height} rows to: {full_path}")
        except Exception as e: print(f"❌ Write failed: {e}")

# 3. Project Helper Functions
def setup_database_environment(db_path, fresh_start=False):
    """
    Initializes DuckDB. Uses RENAME strategy for reliable Fresh Start.
    """
    db_path = Path(db_path).resolve()
    db_path.parent.mkdir(parents=True, exist_ok=True)
    
    # 1. Handle Fresh Start via RENAME (Avoids Lock Issues)
    if fresh_start and db_path.exists():
        print(f"🧹 Fresh Start: Resetting {db_path.name}...")
        gc.collect() # Garbage collect old connections
        
        trash_path = db_path.with_suffix(".duckdb.old")
        if trash_path.exists():
            try: trash_path.unlink() 
            except: pass
            
        try:
            shutil.move(str(db_path), str(trash_path))
            if db_path.with_suffix(".duckdb.wal").exists(): db_path.with_suffix(".duckdb.wal").unlink()
            print("   ✓ Old database moved to trash (Connection Reset).")
        except Exception as e:
            print(f"❌ Warning: Could not move old DB: {e}. Attempting direct overwrite.")

    # 2. Connect
    con = DuckDBWrapper(duckdb_path=db_path)
    con.connect()
    print(f"🔌 Connected to: {db_path}")
    return con

def download_and_cache_data(file_list, base_url, data_dir):
    data_dir = Path(data_dir); data_dir.mkdir(parents=True, exist_ok=True)
    paths, names = [], []
    print("\n🚀 Checking Data Assets...")
    for filename in file_list:
        local_path = data_dir / filename
        url = f"{base_url}/{filename}"; table_name = Path(filename).stem
        if local_path.exists() and local_path.stat().st_size > 0:
            print(f"📂 Cached: '{table_name}'"); paths.append(local_path); names.append(table_name); continue
        print(f"⬇️  Downloading '{filename}'...")
        for attempt in range(1, 4):
            try:
                with requests.get(url, stream=True, headers={'Connection': 'close'}, timeout=(10, 60)) as r:
                    r.raise_for_status()
                    with open(local_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192): f.write(chunk)
                print(f"✅ Saved to {local_path}"); paths.append(local_path); names.append(table_name); time.sleep(1); break
            except Exception as e:
                if local_path.exists(): local_path.unlink()
                if attempt < 3: time.sleep(2)
                else: print(f"❌ Failed {filename}: {e}")
    return paths, names

def process_local_files(file_list):
    """
    Scans for local files and prepares them for registration.
    Args:
        file_list: List of file paths (strings).
    Returns:
        paths (list[Path]), names (list[str])
    """
    paths = []
    names = []
    print("\n🔍 Scanning Local Files...")
    for f in file_list:
        p = Path(f).resolve()
        if p.exists() and p.is_file():
            print(f"✅ Found: {p.name}")
            paths.append(p)
            names.append(p.stem) # 'data.csv' -> 'data'
        else:
            print(f"❌ Not Found: {f}")
    return paths, names

def run_project_sql_pipeline(con, sql_folder, export_folder, output_format="parquet"):
    """Runs all .sql files, exports, and registers. Defaults to Parquet."""
    sql_dir = Path(sql_folder); export_dir = Path(export_folder)
    print(f"\n🚀 Starting SQL Pipeline (Output: {output_format})...")
    for sql_path in sorted(sql_dir.glob("*.sql")):
        print(f"▶︎ Running {sql_path.name}...")
        try: con.export(sql_path.read_text(), sql_path.stem, output_format, export_dir)
        except Exception as e: print(f"❌ Failed {sql_path.name}: {e}")
    extension = f".{output_format}"
    result_paths = sorted(export_dir.glob(f"*{extension}"))
    if result_paths:
        con.register_data_view(result_paths, [p.stem for p in result_paths])
        print(f"✓ Pipeline finished. {len(result_paths)} processed tables registered.")
    else: print("⚠️ Pipeline finished but no output files were found.")

print("✅ System Loaded: Happy Hacking.")

✅ System Loaded: Happy Hacking.


In [5]:
# Set Up your DuckDB Warehouse and Download Datasets

FRESH_START = True  # Set to True to wipe DB and start clean
DB_PATH     = "../data/duckdb/test.duckdb"
DATA_DIR    = "../data/opendata"
BASE_URL    = "https://fastopendata.org/dssg-safestreets"

# Define the files you want to download/load
TARGET_FILES = [
    "nyc_speed_cameras_historic.parquet",
    "test1_nyc_speed_cameras.json",
    "test2_nyc_speed_cameras.csv",
    "test3_nyc_speed_cameras.csv",
    "nyc_traffic_violations_historic.parquet",
    "test1_nyc_traffic_violations.json",
    "test2_nyc_traffic_violations.csv",
    "test3_nyc_traffic_violations.csv"
]

# --- 🚀 EXECUTION ---
# 1. Initialize Database
con = setup_database_environment(DB_PATH, fresh_start=FRESH_START)

# 2. Download Data & Register Views
file_paths, table_names = download_and_cache_data(TARGET_FILES, BASE_URL, DATA_DIR)
con.register_data_view(file_paths, table_names)

# 3. Show what we have
con.show_tables()

🧹 Fresh Start: Resetting test.duckdb...
   ✓ Old database moved to trash (Connection Reset).
🔌 Connected to: /home/erikbrown01/stop-super-speeders-hackathon/data/data/duckdb/test.duckdb

🚀 Checking Data Assets...
📂 Cached: 'nyc_speed_cameras_historic'
📂 Cached: 'test1_nyc_speed_cameras'
📂 Cached: 'test2_nyc_speed_cameras'
📂 Cached: 'test3_nyc_speed_cameras'
📂 Cached: 'nyc_traffic_violations_historic'
📂 Cached: 'test1_nyc_traffic_violations'
📂 Cached: 'test2_nyc_traffic_violations'
📂 Cached: 'test3_nyc_traffic_violations'


               📂 Database Assets               
╭─────────────────────────────────┬────────────╮
│ table_name                      │ table_type │
├─────────────────────────────────┼────────────┤
│ nyc_speed_cameras_historic      │ VIEW       │
├─────────────────────────────────┼────────────┤
│ nyc_traffic_violations_historic │ VIEW       │
├─────────────────────────────────┼────────────┤
│ test1_nyc_speed_cameras         │ VIEW       │
├─────────────────────────────────┼────────────┤
│ test1_nyc_traffic_violations    │ VIEW       │
├─────────────────────────────────┼────────────┤
│ test2_nyc_speed_cameras         │ VIEW       │
├─────────────────────────────────┼────────────┤
│ test2_nyc_traffic_violations    │ VIEW       │
├─────────────────────────────────┼────────────┤
│ test3_nyc_speed_cameras         │ VIEW       │
├─────────────────────────────────┼────────────┤
│ test3_nyc_traffic_violations    │ VIEW       │
╰─────────────────────────────────┴────────────╯

# Queries

In [6]:
# Query with SQK, show_results=True enables scrollable table
query = """



SELECT * FROM nyc_speed_cameras_historic limit 1000




"""
historic_df = con.run_query(query, show_results=False).to_pandas()

In [7]:
# Query with SQK, show_results=True enables scrollable table
query = """



SELECT * FROM test2_nyc_traffic_violations limit 1000




"""
upload_df = con.run_query(query, show_results=False).to_pandas()

# Client

In [10]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Date Transformation

In [11]:
# --- 1. Define the Pydantic Schema for the Transformation Rule ---
class DateParsingRule(BaseModel):
    columns_used: List[str] = Field(
        ..., 
        description="The exact list of column names needed to construct the date."
    )
    join_separator: str = Field(
        " ", 
        description="The separator to use if joining multiple columns (default to space ' '). If single column, this is ignored."
    )
    source_strptime_format: str = Field(
        ..., 
        description="The Python strptime format string matching the source data (e.g., '%Y-%m-%d' or '%B %d, %Y'). If multiple columns, this format must match the joined string."
    )

In [ ]:
example_rows = upload_df.head(10).to_markdown()
columns_available = list(upload_df.columns)
target_format_description = historic_df["issue_date"][0]  # Assume we want to match the first date column

In [14]:
print("\nExample Rows from Upload Data:\n", example_rows)
print("\nAvailable Columns:", columns_available)
print("\nTarget Date Format Example:", target_format_description)


Example Rows from Upload Data:
 |    | License Id   | County      |   Age | Violation Code                    |   Violation Year |   Violation Month |   Points |   Birth Year |   Birth Month |
|---:|:-------------|:------------|------:|:----------------------------------|-----------------:|------------------:|---------:|-------------:|--------------:|
|  0 | BR425170     | Westchester |    70 | SPEED IN ZONE 21-30               |             2025 |                11 |        6 |         1954 |            12 |
|  1 | PK119730     | Other       |    30 | CHILD SAFETY RESTRAINT            |             2025 |                11 |        3 |         1994 |            12 |
|  2 | JW166375     | Nassau      |    19 | SPEED IN ZONE 31-40               |             2025 |                11 |        8 |         2005 |            12 |
|  3 | UK266096     | Richmond    |    43 | FAILED TO STOP SCHOOL BUS         |             2025 |                11 |        5 |         1981 |            12 |
|

In [15]:
# --- 3. Generate the Transformation Rule ---
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"You are a Python Data Engineer. I have a dataset with the following columns: {columns_available}\nHere are the first few rows: {example_rows}\nI need to transform the data into this format: {target_format_description}.\nIdentify the columns that contain date information and define a parsing rule.\nIf the date is split across multiple columns, specify the order they should be joined and the format string to parse the result.",
    config={
        "response_mime_type": "application/json",
        "response_json_schema": DateParsingRule.model_json_schema(),
    },
)

rule = DateParsingRule.model_validate_json(response.text)

print(f"LLM Logic Identified:")
print(f"Columns: {rule.columns_used}")
print(f"Format: '{rule.source_strptime_format}'")



LLM Logic Identified:
Columns: ['Violation Year', 'Violation Month']
Format: '%Y–%m'


In [17]:
# --- 4. APPLY the Transformation (The "ETL" part) ---
# This function uses the LLM's output to actually process the data
def transform_row(row, rule: DateParsingRule):
    try:
        # 1. Extract values based on the columns the LLM identified
        values = [str(row[col]) for col in rule.columns_used]
        
        # 2. Join them (e.g. "2023 October 25")
        raw_date_string = rule.join_separator.join(values)
        
        # 3. Parse using the LLM's format string
        dt_object = datetime.strptime(raw_date_string, rule.source_strptime_format)
        
        # 4. Return in the target format (ISO 8601)
        return dt_object.strftime("%Y-%m-%d")
    except Exception as e:
        return None

# Test on the data
print("\n--- Transformed Data ---")
for row in upload_df.head(10).to_dict(orient="records"):
    new_date = transform_row(row, rule)
    print(f"Original: {row} -> Transformed Date: {new_date}")


--- Transformed Data ---
Original: {'License Id': 'BR425170', 'County': 'Westchester', 'Age': 70, 'Violation Code': 'SPEED IN ZONE 21-30', 'Violation Year': 2025, 'Violation Month': 11, 'Points': 6, 'Birth Year': 1954, 'Birth Month': 12} -> Transformed Date: 2025-11-01
Original: {'License Id': 'PK119730', 'County': 'Other', 'Age': 30, 'Violation Code': 'CHILD SAFETY RESTRAINT', 'Violation Year': 2025, 'Violation Month': 11, 'Points': 3, 'Birth Year': 1994, 'Birth Month': 12} -> Transformed Date: 2025-11-01
Original: {'License Id': 'JW166375', 'County': 'Nassau', 'Age': 19, 'Violation Code': 'SPEED IN ZONE 31-40', 'Violation Year': 2025, 'Violation Month': 11, 'Points': 8, 'Birth Year': 2005, 'Birth Month': 12} -> Transformed Date: 2025-11-01
Original: {'License Id': 'UK266096', 'County': 'Richmond', 'Age': 43, 'Violation Code': 'FAILED TO STOP SCHOOL BUS', 'Violation Year': 2025, 'Violation Month': 11, 'Points': 5, 'Birth Year': 1981, 'Birth Month': 12} -> Transformed Date: 2025-11-01

# Column Mapping

In [ ]:
# --- 1. Define the Pydantic Schema ---
class ColumnPair(BaseModel):
    historical_column: str = Field(
        description="The exact column name from the historical data."
    )
    new_upload_column: Optional[str] = Field(
        description="The matching column name from the new upload. None if no match found."
    )

class MappingResult(BaseModel):
    mappings: List[ColumnPair] = Field(
        description="A list of mappings covering every column in the historical dataset."
    )

In [ ]:
prompt = f"""
    You are a data engineering assistant. You are tasked with mapping the output from a new file to historical data. 
    Below are the first 10 rows of the historical data, and the first 10 rows of the new uploaded data. 
    Based on these rows print a dictionary where the key is the column name from the historical data, 
    and the values are the column names from new upload. Each column from the historical data should appear in the dictionary. 
    If a column from the historical data does not have a match then its value should be None. 
    Every column from the new uploaded data does not need a to appear in the final dictionary. 
    If a column from the new upload does not have a match in the historical data, it should not appear in the final dictionary. 
    Only print the dictionary as the final output. 
"""

requirments = """"
    Requirements:
    1. Return a list of mappings.
    2. Every column from the 'Historical Columns' list MUST appear in the output.
    3. If a historical column has no match in the new file, the value must be None.
"""

In [ ]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# --- 3. Generate Content ---
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"{prompt}\n Example of historical columns:\n {historic_df.head(10).to_string()}\n Example of upload columns:\n {upload_df.head(10).to_string()}\n {requirments}",
    config={
        "response_mime_type": "application/json",
        "response_json_schema": MappingResult.model_json_schema(),
    },
)

In [61]:
response.text

'{"mappings": [{"historical_column": "issue_date", "new_upload_column": "issue_date"}, {"historical_column": "created_at", "new_upload_column": "created_at"}, {"historical_column": "amount_due", "new_upload_column": "amount_due"}, {"historical_column": "county", "new_upload_column": "county"}, {"historical_column": "fine_amount", "new_upload_column": "fine_amount"}, {"historical_column": "interest_amount", "new_upload_column": "interest_amount"}, {"historical_column": "issuing_agency", "new_upload_column": "issuing_agency"}, {"historical_column": "judgment_entry_date", "new_upload_column": "judgment_entry_date"}, {"historical_column": "license_type", "new_upload_column": "license_type"}, {"historical_column": "payment_amount", "new_upload_column": "payment_amount"}, {"historical_column": "penalty_amount", "new_upload_column": "penalty_amount"}, {"historical_column": "plate", "new_upload_column": "plate"}, {"historical_column": "precinct", "new_upload_column": "precinct"}, {"historical_